In [1]:
import pandas as pd
import numpy as np
from os.path import dirname, abspath
from datetime import datetime

d = dirname(dirname(abspath("preprocessing.ipynb")))
df = pd.read_csv(d + '/data/Full_Concat.csv')

df1 = df.drop(columns=['start station id', 'end station id', 'bikeid','Unnamed: 0'])
df1.head()

,tripduration,starttime,stoptime,start station name,start station latitude,start station longitude,end station name,end station latitude,end station longitude,usertype,birth year,gender
0,142,2019-02-01 15:35:02.0820,2019-02-01 15:37:24.1360,Exchange Place,40.716247,-74.033459,Harborside,40.719252,-74.034234,Subscriber,1963.0,1
1,223,2019-02-01 17:00:46.8900,2019-02-01 17:04:30.5500,Exchange Place,40.716247,-74.033459,Grand St,40.715178,-74.037683,Subscriber,1992.0,2
2,106,2019-02-01 17:08:01.3260,2019-02-01 17:09:47.4400,Exchange Place,40.716247,-74.033459,Paulus Hook,40.714145,-74.033552,Subscriber,1960.0,1
3,370,2019-02-01 17:09:31.2100,2019-02-01 17:15:41.6550,Exchange Place,40.716247,-74.033459,Newark Ave,40.721525,-74.046305,Subscriber,1976.0,1
4,315,2019-02-01 17:19:53.2490,2019-02-01 17:25:09.1400,Exchange Place,40.716247,-74.033459,Manila & 1st,40.721651,-74.042884,Subscriber,1980.0,1


In [2]:
# new data frame with split value columns 
starts = df1['starttime'].str.split(" ", n = 1, expand = True) 
ends = df1['stoptime'].str.split(" ", n = 1, expand = True) 

  
# making separate last name column from new data frame 
df1['Start Time']= starts[1] 
df1['End Time']= ends[1] 
  
# Dropping old Name columns 
df1.drop(columns =['starttime','stoptime'], inplace = True) 

df1['Start Time'] = df1['Start Time'].map(lambda x: pd.Timestamp(x))
df1['End Time'] = df1['End Time'].map(lambda x: pd.Timestamp(x))

min_time = min(min(df1['Start Time']),min(df1['End Time']))

df1['Start Time'] = df1['Start Time'].map(lambda x: pd.Timedelta(x - min_time))
df1['End Time'] = df1['End Time'].map(lambda x: pd.Timedelta(x - min_time))

df1['Start Time'] = df1['Start Time'].dt.total_seconds()
df1['End Time'] = df1['End Time'].dt.total_seconds()

#starts[0] = starts[0].map(lambda x: pd.Timestamp(x))
#ends[0] = ends[0].map(lambda x: pd.Timestamp(x))

#starts[0] = starts[0].map(lambda x: pd.to_datetime(x))
#starts[0] = starts[0].dt.to_period('M')
#ends[0] = ends[0].map(lambda x: pd.to_datetime(x))
#ends[0] = ends[0].dt.to_period('M')

#df1['Start Date']= starts[0] 
#df1['End Date']= ends[0] 

#df1['Start Date'] = df1['Start Date'].map(lambda x: pd.Timestamp(x))
#df1['End Date'] = df1['End Date'].map(lambda x: pd.Timestamp(x))

df1.dropna(inplace=True)
#df1 = df.drop(columns=['Start Date', 'End Date'])

df1.head()


,tripduration,start station name,start station latitude,start station longitude,end station name,end station latitude,end station longitude,usertype,birth year,gender,Start Time,End Time
0,142,Exchange Place,40.716247,-74.033459,Harborside,40.719252,-74.034234,Subscriber,1963.0,1,56100.518,56242.572
1,223,Exchange Place,40.716247,-74.033459,Grand St,40.715178,-74.037683,Subscriber,1992.0,2,61245.326,61468.986
2,106,Exchange Place,40.716247,-74.033459,Paulus Hook,40.714145,-74.033552,Subscriber,1960.0,1,61679.762,61785.876
3,370,Exchange Place,40.716247,-74.033459,Newark Ave,40.721525,-74.046305,Subscriber,1976.0,1,61769.646,62140.091
4,315,Exchange Place,40.716247,-74.033459,Manila & 1st,40.721651,-74.042884,Subscriber,1980.0,1,62391.685,62707.576


In [3]:
cat_ftrs = ['start station name', 'end station name', 'gender']
cont_ftrs = ['tripduration','start station longitude', 'end station longitude','start station latitude',
             'end station latitude','Start Time','End Time','birth year']
label = ['usertype']


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# one-hot encoder
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse=False))])

# standard scaler
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])


# label encoder
class ModifiedLabelEncoder(LabelEncoder):
    def fit_transform(self, y, *args, **kwargs):
        return super().fit_transform(y).reshape(-1, 1)
    def transform(self, y, *args, **kwargs):
        return super().transform(y).reshape(-1, 1)

label_transformer = Pipeline(steps=[
    ('label_enc', ModifiedLabelEncoder())])

In [5]:
# collect the transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, cont_ftrs),
        ('cat', categorical_transformer, cat_ftrs),
        ('label',label_transformer, label)])

# fit_transform the data
X_prep = preprocessor.fit_transform(df1)

# little hacky, but collect feature names
feature_names = preprocessor.transformers_[0][-1] + \
                list(preprocessor.named_transformers_['cat'][0].get_feature_names()) + \
                [label]

df_preprocessed = pd.DataFrame(data=X_prep,columns=feature_names)
df_preprocessed.to_csv(d + '/data/Preprocessed_Citi_Bike_Data.csv',index=False)




/Applications/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
type(df_preprocessed)

pandas.core.frame.DataFrame

In [47]:
df_preprocessed.shape

(751299, 249)

In [48]:
df1.shape

(751299, 12)